# CME241 (Winter 2021) -- Assignment 9

## Question 1

In [1]:
from dataclasses import dataclass, field

import numpy as np

from rl.chapter9.order_book import OrderBook as OB
from rl.markov_process import MarkovProcess
from rl.distribution import SampledDistribution, Constant

In [2]:
@dataclass
class OBDynamics(MarkovProcess[OB]):
    """Model of order book dynamics.
    
    Individual orders are uniformly sampled from a defined range,
    and market/limit orders occur in a given ratio.
    """
    
    init_price: int
    limit_orders_per_market_order: int
    max_order_size: int
    book: OB = field(default=OB([], []), init=False)
        
    def __post_init__(self) -> None:
        """Initialize the order book with some limit orders."""
        for i in range(10):
            buy_price = np.random.randint(
                low=1, 
                high=self.init_price
            )
            _, self.book = self.book.buy_limit_order(
                price=buy_price,
                shares=self.randqty()
            )
            sell_price = np.random.randint(
                low=self.init_price+1, 
                high=2*self.init_price
            )
            _, self.book = self.book.sell_limit_order(
                price=sell_price,
                shares=self.randqty()
            )
            
    def randqty(self) -> int:
        """Convenience method for generating a random order qty."""
        return np.random.randint(
            low=1, 
            high=self.max_order_size+1
        )
    
    @property
    def lopmo(self) -> int:
        """Alias for limit_orders_per_market_order."""
        return self.limit_orders_per_market_order
    
    def transition(self, state: OB) -> SampledDistribution[OB]:
        """Transition probabilities are stacked as follows:
        
            - 1/2 probability of buy/sell
            - limit/market order probability
            - price probabilities
            - qty probabilities
            
        Instead of explicitly computing this, its easier to provide a sampled
        distribution.
        """
        def sampler() -> OB:
            """Sample incoming orders and next order book states."""
            side = np.random.choice([-1, 1])
            do_market_order = np.random.rand() < 1 / (1 + self.lopmo)
            limit_price = self.book.mid_price() - side * np.random.randint(1, 10)
            
            if side == 1:  # buy
                if do_market_order:
                    _, new_book = self.book.buy_market_order(self.randqty())
                    return new_book
                else:
                    _, new_book = self.book.buy_limit_order(
                        price=limit_price,
                        shares=self.randqty()
                    )
                    return new_book
            elif side == -1:  # sell
                if do_market_order:
                    _, new_book = self.book.sell_market_order(self.randqty())
                    return new_book
                else:
                    _, new_book = self.book.buy_limit_order(
                        price=limit_price,
                        shares=self.randqty()
                    )
                    return new_book
            else:
                raise ValueError()
        
        return SampledDistribution(sampler=sampler)
            
    
    def print_book(self) -> None:
        """Pretty print the order book."""
        self.book.pretty_print_order_book()

In [3]:
order_book_mp = OBDynamics(
    init_price=100,
    limit_orders_per_market_order=2,
    max_order_size=50
)

In [4]:
order_book_mp.print_book()


Bids
[DollarsAndShares(dollars=99, shares=4),
 DollarsAndShares(dollars=97, shares=16),
 DollarsAndShares(dollars=82, shares=16),
 DollarsAndShares(dollars=54, shares=22),
 DollarsAndShares(dollars=51, shares=42),
 DollarsAndShares(dollars=40, shares=7),
 DollarsAndShares(dollars=30, shares=50),
 DollarsAndShares(dollars=5, shares=46),
 DollarsAndShares(dollars=4, shares=38),
 DollarsAndShares(dollars=1, shares=1)]

Asks

[DollarsAndShares(dollars=103, shares=45),
 DollarsAndShares(dollars=110, shares=17),
 DollarsAndShares(dollars=119, shares=27),
 DollarsAndShares(dollars=120, shares=40),
 DollarsAndShares(dollars=127, shares=48),
 DollarsAndShares(dollars=160, shares=10),
 DollarsAndShares(dollars=169, shares=6),
 DollarsAndShares(dollars=172, shares=17),
 DollarsAndShares(dollars=184, shares=38),
 DollarsAndShares(dollars=195, shares=11)]



In [5]:
start = Constant(order_book_mp.book)
for i, ob in enumerate(order_book_mp.simulate(start)):
    if i > 10:
        break
    
    print("Step=", i)
    ob.pretty_print_order_book()

Step= 0

Bids
[DollarsAndShares(dollars=99, shares=4),
 DollarsAndShares(dollars=97, shares=16),
 DollarsAndShares(dollars=82, shares=16),
 DollarsAndShares(dollars=54, shares=22),
 DollarsAndShares(dollars=51, shares=42),
 DollarsAndShares(dollars=40, shares=7),
 DollarsAndShares(dollars=30, shares=50),
 DollarsAndShares(dollars=5, shares=46),
 DollarsAndShares(dollars=4, shares=38),
 DollarsAndShares(dollars=1, shares=1)]

Asks

[DollarsAndShares(dollars=103, shares=45),
 DollarsAndShares(dollars=110, shares=17),
 DollarsAndShares(dollars=119, shares=27),
 DollarsAndShares(dollars=120, shares=40),
 DollarsAndShares(dollars=127, shares=48),
 DollarsAndShares(dollars=160, shares=10),
 DollarsAndShares(dollars=169, shares=6),
 DollarsAndShares(dollars=172, shares=17),
 DollarsAndShares(dollars=184, shares=38),
 DollarsAndShares(dollars=195, shares=11)]

Step= 1

Bids
[DollarsAndShares(dollars=99, shares=4),
 DollarsAndShares(dollars=97, shares=16),
 DollarsAndShares(dollars=82, shares=1